In [2]:
import tensorflow as tf
from models.dataset import Dataset
from models.ertsrgan.model_generator import g_ertsrgan
from models.model_espcn import espcn 
from models.utils import scale_1 as scale
from models.metrics import psnr, ssim, rmse

# 

In [3]:
model_name = 'espcn'
#model_name='g_ertsrgan'

if model_name == 'g_ertsrgan':
    model='g_ertsrgan'
if model_name == 'espcn':
    model='espcn'
    
train_dataset_path="datasets/train/2x/dataset.tfrecords"
train_dataset_info_path="datasets/train/2x/dataset_info.txt"

batch_size=32
learning_rate=1e-4
num_epochs=10
steps_per_epochs=100
shuffle_buffer_size=0

train_dataset = Dataset(batch_size,
        train_dataset_path,
        train_dataset_info_path,
        shuffle_buffer_size)

scale_factor = train_dataset.scale_factor
checkpoint_paph="{}{}_{}x/model.ckpt".format("checkpoint/",model_name,scale_factor)

In [12]:
if steps_per_epochs == 0:
    steps_per_epoch = train_dataset.examples_num // batch_size \
        if train_dataset.examples_num % batch_size != 0 else 0
else:
    steps_per_epoch = steps_per_epochs

train_dataset = train_dataset.get_data(num_epochs)
train_batch = train_dataset.map(lambda x0,x1,x2,y: (scale(x1),scale(y)))

In [13]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_paph,
        save_weights_only=True,
        monitor='loss',
        save_freq= 'epoch', 
        mode='min',
        save_best_only=True)

In [14]:
if model_name == 'g_ertsrgan':
    model_2x = g_ertsrgan(scale_factor=scale_factor)
if model_name == 'espcn':
    model_2x = espcn(scale_factor=scale_factor)

opt = tf.keras.optimizers.Adam(learning_rate=learning_rate,clipnorm=1.0)
loss = tf.keras.losses.MeanSquaredError()
model_2x.compile(optimizer=opt, loss=loss,metrics=[psnr,ssim,rmse])

In [18]:
model_2x.load_weights(checkpoint_paph)

for i in range(len(model_2x.layers)):
    if(model_2x.layers[i].name != 'final'):
        model_2x.layers[i].trainable=True
        
model_2x.fit(train_batch,epochs=num_epochs,verbose=0,callbacks=checkpoint,steps_per_epoch=steps_per_epoch)

2021-11-07 03:00:02.716826: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2021-11-07 03:02:39.157063: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


# ESPCN

Without load weights and lr 1e-1 = 7.2917

Without load weights and lr 1e-2 = 28.5661

Without load weights and lr 1e-3 = 38.8638

Without load weights and lr 1e-4 = 27.6628

With load weights 1e-3 and lr 1e-4 tranable all layers = 40.3465

With load weights 1e-3 and lr 1e-4 tranable only top = 40.1645

# g_ertsrgan

Without load weights and lr 1e-1 = 5.8966

Without load weights and lr 1e-2 = 39.8428

Without load weights and lr 1e-3 = 37.7591

Without load weights and lr 1e-4 = 31.5835

With load weights 1e-2 and lr 1e-3 tranable all layers = 37.6888

With load weights 1e-2 and lr 1e-3 tranable only top = 40.5480

With load weights 1e-3 and lr 1e-4 tranable all layer = 41.0610 





## Model 4x

In [4]:
print("weights:", len(model_2x.weights))
print("trainable_weights:", len(model_2x.trainable_weights))
print("non_trainable_weights:", len(model_2x.non_trainable_weights))

NameError: name 'model_2x' is not defined

In [18]:

model_name = 'espcn'
#model_name='g_ertsrgan'

if model_name == 'g_ertsrgan':
    model='g_ertsrgan'
if model_name == 'espcn':
    model='espcn'

train_dataset_path="../data/reds/train/dataset.tfrecords"
train_dataset_info_path="../data/reds/train/dataset_info.txt"
#train_dataset_path="datasets/train/4x/dataset.tfrecords"
#train_dataset_info_path="datasets/train/4x/dataset_info.txt"

batch_size=32
fromScale=2 
learning_rate=1e-3
num_epochs=100
steps_per_epochs=100
shuffle_buffer_size=0

train_dataset = Dataset(batch_size,
        train_dataset_path,
        train_dataset_info_path,
        shuffle_buffer_size)

scale_factor = train_dataset.scale_factor
checkpoint_paph="{}{}_{}x/model.ckpt".format("checkpoint/",model_name,scale_factor)

In [19]:
if steps_per_epochs == 0:
    steps_per_epoch = train_dataset.examples_num // batch_size \
        if train_dataset.examples_num % batch_size != 0 else 0
else:
    steps_per_epoch = steps_per_epochs

train_dataset = train_dataset.get_data(num_epochs)
train_batch = train_dataset.map(lambda x0,x1,x2,y: (scale(x1),scale(y)))

In [20]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_paph,
        save_weights_only=True,
        monitor='loss',
        save_freq= 'epoch', 
        mode='min',
        save_best_only=True)

In [21]:
if model_name == 'g_ertsrgan':
    model_4x = g_ertsrgan(scale_factor=scale_factor)
if model_name == 'espcn':
    model_4x = espcn(scale_factor=scale_factor)

opt = tf.keras.optimizers.Adam(learning_rate=learning_rate,clipnorm=1.0)
loss = tf.keras.losses.MeanSquaredError()
model_4x.compile(optimizer=opt, loss=loss,metrics=[psnr,ssim,rmse])

In [323]:
# Transfer learning

checkpoint_paph_2x="{}{}_{}x/model.ckpt".format("checkpoint/",model_name,2)
model_2x.load_weights(checkpoint_paph_2x)

print(model_2x.name)
for i in range(len(model_2x.layers)):
    #print(model_2x.layers[i].name)
    if(model_2x.layers[i].name != 'final'):
        model_4x.layers[i].set_weights(model_2x.layers[i].get_weights())
        model_4x.layers[i].trainable=True

espcn


In [325]:
# Load weights

model_4x.load_weights(checkpoint_paph)

print(model_4x.name)
for i in range(len(model_4x.layers)):
    #print(model_4x.layers[i].name)
    if(model_4x.layers[i].name != 'final'):
        model_4x.layers[i].trainable=True

espcn


In [22]:
print("weights:", len(model_4x.weights))
print("trainable_weights:", len(model_4x.trainable_weights))
print("non_trainable_weights:", len(model_4x.non_trainable_weights))

weights: 6
trainable_weights: 6
non_trainable_weights: 0


In [ ]:
model_4x.fit(train_batch,epochs=num_epochs,verbose=1,callbacks=checkpoint,steps_per_epoch=steps_per_epoch)

Epoch 1/100


2021-11-08 01:38:48.465029: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


100/100 [==============================] - 30s 292ms/step - loss: 0.0229 - psnr: 19.1489 - ssim: 0.3587 - rmse: 0.1449
Epoch 2/100
100/100 [==============================] - 30s 302ms/step - loss: 0.0077 - psnr: 22.5137 - ssim: 0.5359 - rmse: 0.0877
Epoch 3/100
100/100 [==============================] - 30s 302ms/step - loss: 0.0070 - psnr: 23.1187 - ssim: 0.5800 - rmse: 0.0838
Epoch 4/100
100/100 [==============================] - 31s 306ms/step - loss: 0.0058 - psnr: 24.2481 - ssim: 0.6223 - rmse: 0.0760
Epoch 5/100
100/100 [==============================] - 30s 303ms/step - loss: 0.0063 - psnr: 23.6643 - ssim: 0.6053 - rmse: 0.0797
Epoch 6/100
100/100 [==============================] - 30s 304ms/step - loss: 0.0061 - psnr: 23.9542 - ssim: 0.6331 - rmse: 0.0779
Epoch 7/100
100/100 [==============================] - 31s 305ms/step - loss: 0.0058 - psnr: 23.9875 - ssim: 0.6228 - rmse: 0.0760
Epoch 8/100
100/100 [==============================] - 31s 308ms/step - loss: 0.0057 - psnr: 24

100/100 [==============================] - 30s 303ms/step - loss: 0.0049 - psnr: 25.2573 - ssim: 0.6746 - rmse: 0.0701
Epoch 64/100
100/100 [==============================] - 31s 306ms/step - loss: 0.0052 - psnr: 24.9540 - ssim: 0.6668 - rmse: 0.0719
Epoch 65/100
100/100 [==============================] - 30s 303ms/step - loss: 0.0060 - psnr: 24.2125 - ssim: 0.6333 - rmse: 0.0776
Epoch 66/100
100/100 [==============================] - 30s 303ms/step - loss: 0.0048 - psnr: 25.4578 - ssim: 0.6718 - rmse: 0.0694
Epoch 67/100
100/100 [==============================] - 30s 303ms/step - loss: 0.0050 - psnr: 24.9156 - ssim: 0.6628 - rmse: 0.0709
Epoch 68/100
100/100 [==============================] - 30s 304ms/step - loss: 0.0052 - psnr: 24.6244 - ssim: 0.6643 - rmse: 0.0720
Epoch 69/100
100/100 [==============================] - 30s 303ms/step - loss: 0.0050 - psnr: 25.0133 - ssim: 0.6735 - rmse: 0.0708
Epoch 70/100
100/100 [==============================] - 30s 303ms/step - loss: 0.0051 - p

## ESPCN 
Without transfer learning: 34.6189

With transfer learning, only top tranable, and lr 1e-3: 34.7742

With transfer learning, all layers anable, and lr 1e-3:: 34.8456

With load weight 1e-3, only top tranable, and lr 1e-4: 36.3229

With load weight 1e-3, all layers tranable, and lr 1e-4: 36.6999

With load weight 1e-4, all layers tranable, and lr 1e-5: 37.0996


## g_ertsrgan

Without transfer learning: 28.3252

With transfer learning, only top tranable, and lr 1e-2: 34.1716

With transfer learning, all layers tranable, and lr 1e-2: 37.2684

With load weight 1e-2, only top tranable, and lr 1e-4: 37.5994  